In [13]:
import numpy as np
import os
import matplotlib.pyplot as plt
import copy as cp
from scipy.fftpack import fft
from librosa.feature import mfcc
import librosa 
from tqdm import tqdm

In [25]:
def obtain_signal_slice(insignal, fs, cut_start, cut_stop):
    cut_start_idx = int(fs * cut_start)
    cut_stop_idx = int(fs * cut_stop)
    required_length = cut_stop_idx - cut_start_idx

    # Jeśli nagranie jest za krótkie, dodajemy ciszę
    if len(insignal) < cut_stop_idx:
        print("Uzupełnienie krótkiego nagrania ciszą.")
        # Rozszerzenie sygnału ciszą (zerami) do wymaganego czasu
        insignal = np.pad(insignal, (0, required_length - len(insignal)), 'constant')
    
    return insignal[cut_start_idx:cut_stop_idx]


def draw_spectogram(dataset_path, chosen_fpath):
    cut_start = 0
    cut_stop = 20
    
    joint_path = os.path.join(dataset_path, chosen_fpath)
    audio_data, fs = librosa.load(joint_path, sr=None) 
    
    audio_data = obtain_signal_slice(audio_data, fs, cut_start, cut_stop)
    save_dir = os.path.join('Spectograms', os.path.dirname(chosen_fpath))  
    os.makedirs(save_dir, exist_ok=True) 
    save_path = os.path.join(save_dir, os.path.splitext(os.path.basename(chosen_fpath))[0] + '.png')

    S = np.abs(librosa.stft(audio_data)) 
    fig, ax = plt.subplots()
    img = librosa.display.specshow(librosa.amplitude_to_db(S, ref=np.max),y_axis='log', x_axis='time', ax=ax)
    ax.set_title('')
    ax.axis('off')
    plt.savefig(save_path, bbox_inches='tight', pad_inches=0, transparent=True)
    plt.close()

In [3]:
def process_audio_files(root_folder = 'data'):
    audio_formats = {'.wav', '.mp3'}
    dataset_path = os.path.join(root_folder, 'xeno-canto')

    for root, _, files in os.walk(dataset_path):
        for file in files:
            if os.path.splitext(file)[1].lower() in audio_formats:
                try:
                    draw_spectogram(dataset_path, os.path.relpath(os.path.join(root, file), dataset_path))
                except Exception as e:
                    print(f'Error while processing {file}: {e}')
            


In [ ]:
process_audio_files()